In [0]:
#needed libraries 

import numpy as np
import matplotlib.pyplot as plt
import random as rand
import time 

In [0]:
#Global variables 

#environment robot lives in is L by H 
L = 5 #max size of x
H = 6 #max size of y

p_e = 0.01 
gamma = 0.9 

#class states defines the robot state 
#the robot takes in an x,y coordinate for its position 
#this was made into an object in order to create a map of state to action
class state: 
  def __init__(self, x, y): 
    self.x = x
    self.y = y

#class action defines the robot's action 
#it contains an integer for move 
#global definitions for all moves (to avoid confusion)
left = 0 
right = 1 
up = 2 
down = 3 
none = 4
obstacle = 5
class action: 
  def __init__(self, move):
    self.move = move
  #move can be: left, right, up, down, none

In [0]:
def get_probability (s, a, n_s): 

    # probability of staying still if choosing to not move is 1
    if (a == none):
      if s.x == n_s.x and s.y == n_s.y:
        return 1
      else:
        return 0 

    # probability of moving to intended direction that is within bounds is 1 - Pe + Pe/4
    # probability of moving to unintended direction that is within bounds is Pe - Pe/4
    elif ( s.x - 1 == n_s.x and s.y == n_s.y):
      if (a == left):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)
    elif (s.x + 1 == n_s.x and s.y == n_s.y): 
      if (a == right):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)
    elif ( s.x  == n_s.x and s.y + 1 == n_s.y):
      if (a == up):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)
    elif (s.x  == n_s.x and s.y - 1 == n_s.y):
      if (a == down):
        return (1 - p_e + (p_e/4))
      else: 
        return (p_e / 4)

        
    # probability of staying still in non-corner/non-corridor space when moving towards
    # a wall/obstacle is 1 - Pe + Pe/4
    elif (s.x == n_s.x and s.y == n_s.y) and \
            ((a == left and ((s.x == 0 and 1 < s.y < 5 and s.y != 3) or (s.x == 3 and (s.y == 3 or s.y == 1)))) or
              (a == right and s.x == 4 and 0 < s.y < 5) or
              (a == up and 0 < s.x < 4 and s.y == 5) or
              (a == down and ((s.x == 3 and s.y == 0) or ((s.x == 1 or s.x == 2) and s.y == 4)))):
        return (1 - p_e + (p_e/4))

    # probability of staying still when moving away from a wall/obstacle is Pe - Pe/4
    elif (s.x == n_s.x and s.y == n_s.y) and \
            (((s.x == 0 and 1 < s.y < 5 and s.y != 3) or (s.x == 3 and (s.y == 3 or s.y == 1))) or
              (s.x == 4 and 0 < s.y < 5) or
              (0 < s.x < 4 and s.y == 5) or
              ((s.x == 3 and s.y == 0) or ((s.x == 1 or s.x == 2) and s.y == 4))):
        return (p_e / 4)

    # probability of staying still in a corner/corridor while moving towards
    # wall/obstacle is 1 - Pe + Pe/2
    elif (s.x == n_s.x and s.y ==  n_s.y) and \
            ((a == left and s.x == 0) or
              (a == right and ((s.x == 0 and (s.y == 1 or s.y == 3)) or s.x == 4)) or
              (a.move == up and (((s.x == 1 or s.x == 2) and (s.y == 0 or s.y == 2)) or s.y == 5)) or
              (a == down and (((s.x == 1 or s.x == 2) and s.y == 2) or s.y == 0))):
        return (1 - p_e + (p_e / 2))

    # probability of staying still in a corner while moving away from wall is Pe - Pe/2
    elif s.x ==  n_s.x and s.y ==  n_s.y and (s.x == 0 or s.x == 4 or s.y == 5 or s.y == 0):
        return (p_e / 2)
    
    # probability of transitioning to yellow blocks is zero so remains zero
    elif (n_s.y == 1 or n_s.y == 3) and (n_s.x == 1 or n_s.x == 2):
        return 0

    # all other motions(pretty much teleportation) are impossible so remain zero
    else:
        return 0




In [0]:
def get_reward(s):
  if (s.x == 2) and (s.y == 0): 
    return 10 
  elif (s.x == 2) and (s.y == 2):
    return 1
  elif(s.x == 4):
    return -100
  else: 
    return 0 

In [0]:
def get_pi_initial():
  pi_initial = np.zeros((L,H))
  pi_initial[1,1] = obstacle
  pi_initial[2,1] = obstacle
  pi_initial[1,3] = obstacle
  pi_initial[2,3] = obstacle
  return pi_initial

In [74]:
def display_policy(pi):
  for y in reversed(range(H)):
    for x in range(L):
      if pi[x][y]==left:
        print("L ", end=" ")
      elif pi[x][y]==right:
        print("R ", end=" ")
      elif pi[x][y]==up:
        print("U ", end=" ")
      elif pi[x][y]==down:
        print("D ", end=" ")
      elif pi[x][y]==none: 
        print("N ", end=" ")
      elif pi[x][y]==obstacle:
        print("- ", end=" ")
    print("\n")
pi_initial = get_pi_initial()
display_policy(pi_initial)

L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  



In [0]:
def am_i_obstacle(s):
  if (s.x==1):
    if((s.y==3) or (s.y==1)):
      return 1;
  elif (s.x==2):
    if((s.y==3) or (s.y==1)):
      return 1;
  if ((s.x < 0) or (s.y < 0) or (s.x >= L) or (s.y >= H)):
    return 1; 
  return 0;  

def get_value(x, y, values, a):
  
  s = state(x, y) 

  if am_i_obstacle(s)==1:
    return 0

  new_value = 0 
  

  go_left = state(x-1,y)
  if am_i_obstacle(go_left) == 0: 
    new_value += get_probability(s, a, go_left)*(get_reward(go_left)+gamma*values[x-1][y])
      
  go_right = state(x+1,y) 
  if am_i_obstacle(go_right) == 0:
    new_value+= get_probability(s, a, go_right)*(get_reward(go_right)+gamma*values[x+1][y])

  go_up = state(x,y+1) 
  if am_i_obstacle(go_up) == 0:
    new_value+= get_probability(s, a, go_up)*(get_reward(go_up)+gamma*values[x][y+1])

  go_down = state(x,y-1) 
  if am_i_obstacle(go_down) == 0:
    new_value+= get_probability(s, a, go_down)*(get_reward(go_down)+gamma*values[x][y-1])
      
  new_value += get_probability(s, a, s)*(get_reward(s)+gamma*values[x][y])

  return new_value

def policy_evaluation(values, pi):
  
  for x in range (L): 
    for y in range (H): 
      values[x][y]=get_value(x, y, values, pi[x][y])
  return values




In [77]:
def policy_onestep_lookahead (values, pi):
  for x in range(L):
    for y in range(H): 

      if (pi[x][y] == obstacle):
        continue
    
      left_value = get_value(x, y, values, left)
      right_value = get_value(x, y, values, right)
      up_value = get_value(x, y, values, up)
      down_value = get_value(x, y, values, down)
      none_value = get_value(x, y, values, none)

      pi[x][y] = np.argmax([left_value, right_value, up_value, down_value, none_value])

  return pi 

values_initial = np.zeros((L,H))
pi_initial = get_pi_initial()
pi_1 = policy_onestep_lookahead(values_initial, pi_initial)
display_policy(pi_1)
#print(pi_1)

L  L  L  N  L  

L  L  L  N  L  

L  -  -  N  L  

L  R  N  L  L  

L  -  -  N  L  

L  R  N  L  L  



In [80]:
def policy_iteration(): 
  pi = get_pi_initial() 
  values = np.zeros((L,H))

  while 1: 
    value_previous = np.linalg.norm(values)
    values = policy_evaluation(values, pi)
    pi = policy_onestep_lookahead(values, pi)
    next_value = np.linalg.norm(values)
    if np.abs(next_value-value_previous) < 1e-10:
      break

  display_policy(pi)
  
policy_iteration()


D  D  D  D  L  

D  L  R  D  L  

D  -  -  D  L  

D  R  R  D  L  

D  -  -  D  L  

R  R  N  L  L  



In [0]:
def value_onestep(values):
  for x in range(L): 
    for y in range(H): 
        left_value = get_value(x, y, values, left)
        right_value = get_value(x, y, values, right)
        up_value = get_value(x, y, values, up)
        down_value = get_value(x, y, values, down)
        none_value = get_value(x, y, values, none)

        values[x][y] = np.max([left_value, right_value, up_value, down_value, none_value])
  return values

def grid_create(l, h) :
    grid = np.zeros((l, h))
    return grid

def setup(state_space) :
    state_space[4, :] = -100
    state_space[2, 0] = 10
    state_space[2, 2] = 5
    state_space[1, 3] = state_space[2, 3] = -1 * np.inf
    state_space[1, 1] = state_space[2, 1] = -1 * np.inf
    return state_space

def display_value(values):
  state_space = grid_create(L, H)
  state_space = setup(state_space)
  plt.imshow(state_space.T, origin='lower', extent=[0, L, 0, H])
  value = plt.subplot()
  for i in range(values.shape[0]):
      for j in range(values.shape[1]):
          val = values[i, j]
          if am_i_obstacle(state(i,j)) == 1:
              continue
          value.text(i + 0.35, j + 0.35, '%.2f'%val, color='r')

In [95]:
def value_iteration(): 
  values = np.zeros((L,H))

  while True: 
    previous_value = np.linalg.norm(values)
    values = value_onestep(values)
    next_value = np.linalg.norm(values)
    if np.abs(next_value-previous_value) < 1e-10: 
      break 
  
  pi = get_pi_initial()
  pi = policy_onestep_lookahead(values, pi)
  display_policy(pi)

value_iteration()

D  D  D  D  L  

D  L  R  D  L  

D  -  -  D  L  

D  R  R  D  L  

D  -  -  D  L  

R  R  N  L  L  



In [108]:
def top(new_p_e, new_gamma): 
  global p_e
  p_e = new_p_e 
  global gamma
  gamma = new_gamma 

  print("Running case with error probability %.2f and discount %.2f "% (p_e, gamma))
  print("Pi initial: ")
  pi0 = get_pi_initial()
  display_policy(pi0)

  print("Running Policy Iteration: ")
  policy_iteration()

  print("Running Value Iteration: ")
  value_iteration() 


top(0.01, 0.1)

Running case with error probability 0.01 and discount 0.10 
Pi initial: 
L  L  L  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

L  -  -  L  L  

L  L  L  L  L  

Running Policy Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  L  L  

D  -  -  D  L  

R  R  N  L  L  

Running Value Iteration: 
D  L  L  N  L  

D  L  L  N  L  

D  -  -  N  L  

R  R  N  L  L  

D  -  -  D  L  

R  R  N  L  L  

